In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
from matplotlib import style

from pathlib import Path

from autumn.settings import INPUT_DATA_PATH


If not in an 'AuTuMN' python environment. Edit and run the cell below to manually create the path to files.

In [ ]:
style.use("dark_background")
p = Path(".")
# INPUT_DATA_PATH = p.joinpath('c:\\', "Users","maba0001", "AuTuMN", "data", "inputs")

In [ ]:

fb_2021 = p.joinpath(INPUT_DATA_PATH, "mobility", "movement-range-2021.txt")
fb_2022 = p.joinpath(INPUT_DATA_PATH, "mobility", "movement-range-2022-04-25.txt")




In [ ]:
df_list = []
for file in {fb_2021,fb_2022}:
    df = pd.read_csv(file, delimiter="\t")
    df = df[df["country"]=="BTN"]
    df_list.append(df)

fb_mov = pd.concat(df_list)

Take the average of all sub regions of Bhutan to arrive at national movement values.

In [ ]:
btn_mov = fb_mov.groupby('ds', as_index=False).mean()

In [ ]:
btn_mov.rename(columns={"all_day_bing_tiles_visited_relative_change":"visited_tiles","all_day_ratio_single_tile_users":"single_tiles" }, inplace=True)

Change the values of `MAP_FACOTRS` as required. 

In [ ]:
MAP_FACTORS = {'retail_and_recreation_percent_change_from_baseline': 30,
       'grocery_and_pharmacy_percent_change_from_baseline': 20,
       'parks_percent_change_from_baseline': 20,
       'transit_stations_percent_change_from_baseline': 40,
       'workplaces_percent_change_from_baseline': 50,
       'residential_percent_change_from_baseline': 40}

In [ ]:
for col, val in MAP_FACTORS.items():
    if col != 'residential_percent_change_from_baseline':
        btn_mov[col] = val * btn_mov["visited_tiles"]
    else:
        btn_mov[col] = val * btn_mov["single_tiles"]
        

In [ ]:
btn_mov

In [ ]:
MOB_COL = [col for col in MAP_FACTORS.keys()]

In [ ]:
corr_plot = btn_mov[['visited_tiles', 'single_tiles']].corr()
plt.figure(figsize=(5,5))
hm = sns.heatmap(corr_plot, annot = True)

In [ ]:
btn_mov.set_index(btn_mov['ds'], inplace= True)

In [ ]:
btn_mov[MOB_COL].plot(figsize=(20,6))

In [ ]:
btn_mov[["visited_tiles","single_tiles"]].plot(figsize=(20,6))

Create a csv for input db.

In [ ]:
btn_mov

In [ ]:
input_df = btn_mov[['ds']+MOB_COL]
input_df["sub_region_1"] = None

In [ ]:
#input_df.rename(columns=lambda x:  f"{x}_percent_change_from_baseline" if x not in {'ds',"polygon_name"} else x,inplace=True)
input_df.rename(columns={'ds':"date"}, inplace = True)
input_df["country_region"] = "Bhutan"

In [ ]:
btn_csv = p.joinpath(INPUT_DATA_PATH, "mobility", "BTN_mobility.csv")

In [ ]:
input_df.to_csv(btn_csv, index=False)

Once the CSV is created run<br>
`python -m autumn db build`<br>
`python -m autumn secrets write data/inputs/inputs.secret.db`<br>
Git push the secret hash and the encrpyted db file to master.